In [4]:
pip install geopandas

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... error
  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [28 lines of output]
      Traceback (most recent call last):
        File "/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/opt/anaconda3/envs/myenv/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 149, in prepare_metadata_for_build_wheel
          return hook(metadata_directory, config_settings)
    

In [8]:
import pandas as pd
import numpy as np

def load_country_data():
    """
    Simulate loading data for countries.
    Replace with actual data loading from your files in GPT knowledge.
    """
    data = {
        "russian_imports": pd.read_excel("Reliance on Russian imports - July 2023.xlsx"),
        "lng_data": pd.read_excel("LNG plot data 2024-11-07.xlsx"),
        "gas_storage": pd.read_excel("Weekly Storage EU & UA 2024-11-28.xlsx"),
        "natural_gas_prices": pd.read_csv("Natural Gas Futures Historical Data (1).csv"),
    }
    return data

def calculate_risk_scores(country, data):
    """
    Calculate geopolitical risk scores for a specific country based on various metrics.
    """
    # Russian import dependency (scale 0–1)
    russian_dependency = data["russian_imports"].set_index("Country").loc[country, "Import Dependency"] / 100

    # Gas storage levels (inverse risk)
    avg_storage = data["gas_storage"].set_index("Country").loc[country, "Average Storage Level"]
    storage_risk = 1 - avg_storage / 100  # Higher levels reduce risk

    # LNG import trends (scale 0–1)
    lng_imports = data["lng_data"].set_index("Country").loc[country, "Import Trend"]
    lng_risk = 1 - lng_imports / max(data["lng_data"]["Import Trend"])  # Normalize

    # Natural gas price volatility (scale 0–1)
    price_volatility = data["natural_gas_prices"]["Price"].pct_change().std()
    price_risk = min(price_volatility / 0.05, 1)  # Cap max risk for extreme volatility

    # Weighted risk calculation
    risk_score = (
        0.4 * russian_dependency
        + 0.3 * storage_risk
        + 0.2 * lng_risk
        + 0.1 * price_risk
    )
    return risk_score

def normalize_scores(risk_scores):
    """
    Normalize risk scores to a 0–5 scale for easier interpretation.
    """
    min_score = min(risk_scores.values())
    max_score = max(risk_scores.values())
    normalized_scores = {
        country: 5 * (score - min_score) / (max_score - min_score) for country, score in risk_scores.items()
    }
    return normalized_scores

def generate_country_dashboard(country, data):
    """
    Generate a dashboard for a specific country with risk metrics.
    """
    russian_dependency = data["russian_imports"].set_index("Country").loc[country, "Import Dependency"]
    avg_storage = data["gas_storage"].set_index("Country").loc[country, "Average Storage Level"]
    lng_imports = data["lng_data"].set_index("Country").loc[country, "Import Trend"]
    price_volatility = data["natural_gas_prices"]["Price"].pct_change().std()

    dashboard = f"""
    **Country: {country}**
    - Russian Import Dependency: {russian_dependency:.2f}%
    - Average Gas Storage Levels: {avg_storage:.2f}%
    - LNG Import Trends: {lng_imports:.2f}
    - Natural Gas Price Volatility: {price_volatility:.4f}
    """
    return dashboard

def main_country_analysis(prompt):
    """
    Main function to handle country-specific geopolitical risk prompts.
    """
    data = load_country_data()
    if prompt.startswith("geopolitical risk for"):
        country = prompt.replace("geopolitical risk for ", "").strip()
        try:
            risk_score = calculate_risk_scores(country, data)
            normalized_score = normalize_scores({country: risk_score})[country]
            dashboard = generate_country_dashboard(country, data)
            return f"{dashboard}\n\n**Risk Score (0–5): {normalized_score:.2f}**"
        except KeyError:
            return f"Data for {country} not found. Please check the input."
    elif prompt == "country risk dashboard":
        risk_scores = {country: calculate_risk_scores(country, data) for country in data["russian_imports"]["Country"]}
        normalized_scores = normalize_scores(risk_scores)
        dashboard = "\n".join(
            [f"**{country}: {score:.2f}**" for country, score in normalized_scores.items()]
        )
        return f"**Country Risk Dashboard:**\n{dashboard}"
    else:
        return "Command not recognized. Try 'geopolitical risk for [country]' or 'country risk dashboard'."

# Example usage
# prompt = "geopolitical risk for Germany" or "country risk dashboard"
# result = main_country_analysis(prompt)
# Display result in GPT chat